In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
from context import Context
from contracts.farm_contract import FarmContract
from contracts.fees_collector_contract import FeesCollectorContract
from utils.utils_generic import log_warning, log_step_pass, log_step_fail
context = Context()
fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]
log_step_pass(f"Fees collector contract: {fees_collector_contract.address}")

Farm rewards distribution

In [ ]:
import json

rewards_config_file = "emission_rewards_24_12_12.json"
log_warning(f"Reading rewards config from {rewards_config_file}")

# load the rewards json file
with open(rewards_config_file) as f:
    emission_rewards = json.load(f)

In [ ]:
from tools.runners.farm_runner import get_farm_addresses_from_chain
from utils.utils_generic import get_continue_confirmation
from utils.utils_chain import get_account_key, string_to_hex

# set the rewards
farm_addresses = get_farm_addresses_from_chain("v2")
log_step_pass(f"Retrieved {len(farm_addresses)} farm addresses.")
if get_continue_confirmation():
    for farm_address in farm_addresses:
        try:
            farm_contract: FarmContract
            farm_contract = FarmContract.load_contract_by_address(farm_address)
            log_step_pass(f"Setting rewards for farm {farm_contract.farmToken} {farm_contract.address}")
        except AttributeError as e:
            log_step_fail(f"Failed to load farm contract {farm_address}: {e}")
            if not get_continue_confirmation():
                break
            continue
        
        if farm_contract.farmToken not in emission_rewards:
            current_rpb = farm_contract.get_per_block_reward_amount(context.network_provider.proxy)
            log_step_fail(f"Farm token {farm_contract.farmToken} not found in rewards config. \nCurrent rewards: {current_rpb:,}. \nSkipping.")
            continue

        new_rewards = emission_rewards[farm_contract.farmToken] * 10**18
        log_warning(f"New rewards: {new_rewards:,}")

        if not new_rewards:
            new_rewards = 1     # can't set 0 rpb
            # check if rewards producing is stopped, otherwise offer to stop it
            rewards_enabled = get_account_key(farm_address, string_to_hex("produce_rewards_enabled"), context.network_provider.proxy)
            if rewards_enabled == "01":
                log_warning(f"Produce rewards enabled for farm {farm_contract.farmToken}. Want to stop?")
                if get_continue_confirmation():
                    tx_hash = farm_contract.end_produce_rewards(context.deployer_account, context.network_provider.proxy)
                    if not context.network_provider.check_simple_tx_status(tx_hash, f"stop rewards for: {farm_address}"):
                        if not get_continue_confirmation():
                            break
                
        # if not get_continue_confirmation():
        #      break
        current_rpb = farm_contract.get_per_block_reward_amount(context.network_provider.proxy)
        if current_rpb == new_rewards:
            log_warning(f"Rewards for farm {farm_contract.farmToken} are already set to {new_rewards:,}. Skipping.")
            continue

        tx_hash = farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, new_rewards)
        if not context.network_provider.check_simple_tx_status(tx_hash, f"set rewards per block for: {farm_address}"):
                if not get_continue_confirmation():
                    break

log_step_pass("All done.")

Fees collector rewards distribution

In [ ]:
new_rewards = emission_rewards["FEESCOLLECTOR"] * 10**18
log_warning(f"New fees collector rewards: {new_rewards:,}")
tx_hash = fees_collector_contract.set_locked_tokens_per_block(context.deployer_account, context.network_provider.proxy, new_rewards)
context.network_provider.check_simple_tx_status(tx_hash, f"set fees collector rewards per block")